## 什么是数据连接？

LLM应用往往需要用户特定的数据，而这些数据并不属于模型的训练集。`LangChain` 的数据连接概念，通过提供以下组件，实现用户数据的加载、转换、存储和查询：

- 文档加载器：从不同的数据源加载文档
- 文档转换器：拆分文档，将文档转换为问答格式，去除冗余文档，等等
- 文本嵌入模型：将非结构化文本转换为浮点数数组表现形式，也称为向量
- 向量存储：存储和搜索嵌入数据（向量）
- 检索器：提供数据查询的通用接口

## 数据连接实践

在LLM应用连接用户数据时，通常我们会以如下步骤完成：

1. 加载文档
2. 拆分文档
3. 向量化文档分块
4. 向量数据存储

这样，我们就可以通过向量数据的检索器，来查询用户数据。接下来我们看看每一步的代码实现示例。最后，我们将通过一个完整的示例来演示如何使用数据连接。

### 加载文档

`Langchain` 提供了多种文档加载器，用于从不同的数据源加载不同类型的文档。比如，我们可以从本地文件系统加载文档，也可以通过网络加载远程数据。想了解 `Langchain` 所支持的所有文档加载器，请参考[Document Loaders](https://python.langchain.com/docs/integrations/document_loaders/)。

在本节课程中，我们将使用最基本的 `TextLoader` 来加载本地文件系统中的文档。代码如下：

In [3]:
from langchain.document_loaders import TextLoader

loader = TextLoader("./file/demo.txt")
docs = loader.load()

在上述代码中，我们使用 `TextLoader` 加载了本地文件系统中的 `./demo.txt` 文件。`TextLoader` 的 `load` 方法返回一个 `Document` 对象数组（`Document` 是 `Langchain` 提供的文档类，包含原始内容和元数据）。我们可以通过 `Document` 对象的 `page_content` 属性来访问文档的原始内容。

### 拆分文档

拆分文档是文档转换中最常见的操作。拆分文档的目的是将文档拆分为更小的文档块，以便更好地利用模型。当我们要基于长篇文本构建QA应用时，必须将文本分割成块，这样才能在数据查询中，基于相似性找到与问题最接近的文本块。这也是常见的AI问答机器人的工作原理。
`Langchain` 提供了多种文档拆分器，用于将文档拆分为更小的文档块。我们逐个看看这些拆分器的使用方法。

#### 按字符拆分

`CharacterTextSplitter` 是最简单的文档拆分器，它将文档拆分为固定长度的文本块。代码如下：

In [14]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="\n\n",  # 
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
)

split_docs = text_splitter.split_documents(docs)

Created a chunk of size 3321, which is longer than the specified 1000
Created a chunk of size 1108, which is longer than the specified 1000
Created a chunk of size 2873, which is longer than the specified 1000
Created a chunk of size 1539, which is longer than the specified 1000


In [15]:
for i in split_docs:
    print(i)

page_content='第一章 总体要求\n一、组织架构及岗位设置\n证券公司应确定股票期权自营业务的负责部门，并配置期权自营业务团队开展自营业务。自营业务负责部门的职责包括但不限于股票期权自营业务制度与流程制定、日常管理、风险控制等。岗位设置包括但不限于投资经理、交易员、风险控制、运行维护等。各岗位应配备符合条件的专业人员，建议投资经理岗、交易岗等核心岗位专职担任。\n期权自营业务涉及结算、运营管理、信息技术、风险管理、合规管理等多个业务和功能支持部门。支持部门应设立专门的岗位，负责与期权自营业务相关的工作，内容包括但不限于期权自营业务风险管理、内控与合规管理、清算、资金管理、技术支持、财务管理等。\n期权自营业务岗位设置须符合合规及业务隔离要求，主要岗位专岗负责，并建立备岗机制。\n证券公司应当建立完善的股票期权自营业务决策授权体系。决策授权体系包含公司管理层、业务部门两个层面，明确股票期权自营业务决策机构与决策机制，确定合理的股票期权业务规模和可承受的风险限额。\n二、制度与流程\n证券公司股票期权自营部门应制定健全的业务制度和完备的业务流程，涵盖自营业务管理、交易管理、结算、风险管理、内部控制制度、应急预案等各个业务环节，并明确各相关部门的职责和岗位设置。\n证券公司应制定股票期权自营业务管理办法，保障业务规范开展。内容包括但不限于自营业务实施机制、决策流程、授权体系、风险管理、合规与内部控制、考评及问责机制等。\n证券公司应制定股票期权自营业务风险管理制度，加强公司对期权业务的风险管理、有效防范和化解业务风险。内容包括但不限于股票期权自营业务风险管理基本原则、风险指标设计、风险识别与评估、风险控制与处置、风险定期报告机制、压力测试的情景设置和分析等。\n证券公司应制定股票期权自营业务应急预案，以应对期权自营业务中的突发事件，保障期权业务平稳开展。\n三、合规与内控管理\n证券公司应当建立健全股票期权自营业务内部控制制度和隔离墙制度，防范不同主体之间的利益冲突，合规开展业务。\n（一）合规制度\n1、隔离墙制度\n证券公司应建立股票期权自营业务信息隔离墙制度，并将其纳入公司内部控制体系。证券公司应根据自身及行业发展变化情况，对信息隔离墙制度进行持续调整与优化。\n（1）隔离墙制度的内容与范围\n股票期权自营业务与期权经纪、投资咨询、做市业务、投资银行、资

#### 拆分代码

`RecursiveCharacterTextSplitter` 的 `from_language` 函数，可以根据编程语言的特性，将代码拆分为合适的文本块。代码如下：

In [18]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, Language

PYTHON_CODE = """
def getlnglat(address):
    url = "http://api.map.baidu.com/geocoding/v3/"
    output = "json"
    ak = "TmMEiHDnLCl1tHb9yWOMI5gv168XpIv3"
    add = quote(address)  # 由于本文城市变量为中文，为防止乱码，先用quote进行编码
    uri = url + "?" + "address=" + add + "&output=" + output + "&ak=" + ak
    req = urlopen(uri)
    res = req.read().decode()  # 将其他编码的字符串解码成unicode
    temp = json.loads(res)  # 对json数据进行解析
    return temp
"""
python_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON, chunk_size=200, chunk_overlap=0
)
python_docs = python_splitter.create_documents([PYTHON_CODE])
for k in python_docs:
    print(k)

page_content='def getlnglat(address):\n    url = "http://api.map.baidu.com/geocoding/v3/"\n    output = "json"\n    ak = "TmMEiHDnLCl1tHb9yWOMI5gv168XpIv3"\n    add = quote(address)  # 由于本文城市变量为中文，为防止乱码，先用quote进行编码' metadata={}
page_content='uri = url + "?" + "address=" + add + "&output=" + output + "&ak=" + ak\n    req = urlopen(uri)\n    res = req.read().decode()  # 将其他编码的字符串解码成unicode\n    temp = json.loads(res)  # 对json数据进行解析' metadata={}
page_content='return temp' metadata={}


#### Markdown文档拆分

`MarkdownHeaderTextSplitter` 可以将Markdown文档按照段落结构，基于Markdown语法进行文档分块。代码如下：

In [20]:
from langchain.text_splitter import MarkdownHeaderTextSplitter

markdown_document = "# Chapter 1\n\n    ## Section 1\n\nHi this is the 1st section\n\nWelcome\n\n ### Module 1 \n\n Hi this is the first module \n\n ## Section 2\n\n Hi this is the 2nd section"

# 这里可以标记出来那些是头，遇到这些进行切分
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]

splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
splits = splitter.split_text(markdown_document)
for l in splits:
    print(l)

page_content='Hi this is the 1st section  \nWelcome' metadata={'Header 1': 'Chapter 1', 'Header 2': 'Section 1'}
page_content='Hi this is the first module' metadata={'Header 1': 'Chapter 1', 'Header 2': 'Section 1', 'Header 3': 'Module 1'}
page_content='Hi this is the 2nd section' metadata={'Header 1': 'Chapter 1', 'Header 2': 'Section 2'}


#### 按字符递归拆分

这也是对于普通文本的推荐拆分方式。它通过一组字符作为参数，按顺序尝试使用这些字符进行拆分，直到块的大小足够小为止。默认的字符列表是["\n\n", "\n", " ", ""]。它尽可能地保持段落，句子，单词的完整，因此尽可能地保证语义完整。

In [23]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# RecursiveCharacterTextSplitter 使用的是["\n\n", "\n", " ", ""] 尽可能的保证句子单词段落的完整性。
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=100,
    chunk_overlap=20,
    length_function=len,
)
texts = text_splitter.split_documents(docs)
for text in texts:
    print(text)

page_content='第一章 总体要求\n一、组织架构及岗位设置' metadata={'source': './file/demo.txt'}
page_content='证券公司应确定股票期权自营业务的负责部门，并配置期权自营业务团队开展自营业务。自营业务负责部门的职责包括但不限于股票期权自营业务制度与流程制定、日常管理、风险控制等。岗位设置包括但不限于投资经理、交' metadata={'source': './file/demo.txt'}
page_content='控制等。岗位设置包括但不限于投资经理、交易员、风险控制、运行维护等。各岗位应配备符合条件的专业人员，建议投资经理岗、交易岗等核心岗位专职担任。' metadata={'source': './file/demo.txt'}
page_content='期权自营业务涉及结算、运营管理、信息技术、风险管理、合规管理等多个业务和功能支持部门。支持部门应设立专门的岗位，负责与期权自营业务相关的工作，内容包括但不限于期权自营业务风险管理、内控与合规管理、清' metadata={'source': './file/demo.txt'}
page_content='期权自营业务风险管理、内控与合规管理、清算、资金管理、技术支持、财务管理等。' metadata={'source': './file/demo.txt'}
page_content='期权自营业务岗位设置须符合合规及业务隔离要求，主要岗位专岗负责，并建立备岗机制。' metadata={'source': './file/demo.txt'}
page_content='证券公司应当建立完善的股票期权自营业务决策授权体系。决策授权体系包含公司管理层、业务部门两个层面，明确股票期权自营业务决策机构与决策机制，确定合理的股票期权业务规模和可承受的风险限额。' metadata={'source': './file/demo.txt'}
page_content='二、制度与流程\n证券公司股票期权自营部门应制定健全的业务制度和完备的业务流程，涵盖自营业务管理、交易管理、结算、风险管理、内部控制制度、应急预案等各个业务环节，并明确各相关部门的职责和岗位设置。' metadata={'source': './file/demo.txt'}
pa

### 向量化文档分块

`Langchain` 的 `Embeddings` 类实现与文本嵌入模型进行交互的标准接口。当前市场上有许多嵌入模型提供者（如OpenAI、Cohere、Hugging Face等）。

嵌入模型创建文本片段的向量表示。这意味着我们可以在向量空间中处理文本，并进行语义搜索等操作，从而查找在向量空间中最相似的文本片段。

注，文本之间的相似度由其向量表示的欧几里得距离来衡量。欧几里得距离是最常见的距离度量方式，也称为L2范数。对于两个n维向量a和b，欧几里得距离可以通过以下公式计算：

计算公式：d(a, b) = √((a₁ - b₁)² + (a₂ - b₂)² + ... + (aₙ - bₙ)²)
当使用OpenAI的文本嵌入模型时，我们使用如下代码来创建文本片段的向量表示：

In [26]:
from langchain.embeddings import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings()
embeddings = embeddings_model.embed_documents(
    [
        "你好!",
        "Langchain!",
        "你真棒！"
    ]
)
embeddings

[[0.001767348474591444,
  -0.016549955833298362,
  0.009669921232251705,
  -0.024465152668289573,
  -0.04928377577655549,
  0.011336278019518115,
  -0.008211858461316996,
  -0.01088812932786606,
  -0.01795121072190508,
  -0.01324248948210488,
  0.03403408035057112,
  0.0026052610136390753,
  -0.006431885909394602,
  -0.009076597048277575,
  -0.007839453302328075,
  -0.022622059729712433,
  0.028605797878777543,
  -0.01921360355383904,
  0.01179705078850112,
  -0.020703224189794717,
  -0.004421526972980707,
  0.0070062744430335906,
  -1.4916934361990798e-05,
  0.005440908411769653,
  -0.004790776811128811,
  -0.0007215358104482736,
  0.004358407517648521,
  -0.013444471925432386,
  0.007018898054703258,
  -0.02996917960406885,
  0.03249396340529164,
  0.007517543055679111,
  -0.017925962567243183,
  0.001721586776343353,
  0.01372219883274559,
  -0.018519287682539874,
  -0.007536478706014255,
  -0.0011440425726175886,
  0.020198268547137233,
  0.005339917190105899,
  0.01519919725401544

In [46]:
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader

# 加载文件到文本加载器里面
loader = TextLoader("./file/demo.txt")
docs = loader.load()

# 这里选择RecursiveCharacterTextSplitter，适合那种 key：value 标准的知识文档
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=20,
    length_function=len,
)
# 将文本进行切割
texts = text_splitter.split_documents(docs)
# 从向量数据库中搜索文件
db = Chroma.from_documents(texts, OpenAIEmbeddings())
db

In [47]:
# 从向量数据库中搜索需要问问题
query = "什么是即时处置线？"
docs = db.similarity_search(query)
print(docs[0].page_content)

即时处置线：自营账户实时风险值达到100%时，触发即时处置线。当盘中自营账户实时风险值达到或者高于即时处置线时，证券公司可以选择实时对该自营账户进行强行平仓。不做即时平仓的证券公司，应做好相应的资金
